In [23]:
import requests
import yaml
import time
import signal
from collections import defaultdict
import sys

In [24]:
def log_availability(availability):
    domain_availability = defaultdict(lambda: {'total': 0, 'up': 0})
    for url, data in availability.items():
        domain = url.split('/')[2]
        domain_availability[domain]['total'] += data['total']
        domain_availability[domain]['up'] += data['up']

    for domain, data in domain_availability.items():
        if data['total'] == 0:
            percentage = 0
        else:
            percentage = round((data['up'] / data['total']) * 100)
        print(f"{domain} has {percentage}% availability")

In [25]:
# Calculate availability percentage for each endpoint
def check_health(endpoints):
    availability = defaultdict(lambda: {'total': 0, 'up': 0})
    while True:
        for endpoint in endpoints:
            url = endpoint.get('url')
            method = endpoint.get('method', 'GET')
            headers = endpoint.get('headers', {})
            body = endpoint.get('body', None)

            if url:
                try:
                    if method == 'GET':
                        response = requests.get(url, headers=headers, timeout=5)
                    elif method == 'POST':
                        if body is None:
                            body = {}
                        response = requests.post(url, headers=headers, json=body, timeout=5)
                    else:
                        raise ValueError('Invalid HTTP method')

                    if response.status_code >= 200 and response.status_code < 300:
                        response_time = response.elapsed.total_seconds() * 1000
                        if response_time < 500:
                            availability[url]['up'] += 1

                    availability[url]['total'] += 1

                    print(f"Endpoint {url} - Status Code: {response.status_code}")

                except requests.RequestException as e:
                    print(f"Request to {url} failed: {e}")

        # Calculate and print availability percentages for each domain after each cycle
        log_availability(availability)

        # Wait for 15 seconds before starting the next cycle
        time.sleep(15)

In [26]:
def read_config(file_path):
    try:
        with open(file_path, 'r') as file:
            return yaml.safe_load(file)
    except FileNotFoundError:
        print("File not found. Please provide a valid YAML configuration file.")
        sys.exit(1)
    except yaml.YAMLError as exc:
        print(f"Error in YAML file: {exc}")
        sys.exit(1)

In [27]:
def exit_gracefully(signum, frame):
    print("\nExiting...")
    sys.exit(0)

In [28]:
if __name__ == "__main__":
    # if len(sys.argv) != 2:
    #     print("Usage: python health_checker_script.py <config_file.yaml>")
    #     sys.exit(1)

    # file_path = sys.argv[1]
    endpoints = read_config("config.yaml")

    signal.signal(signal.SIGINT, exit_gracefully)

    check_health(endpoints)

Endpoint https://fetch.com/ - Status Code: 200
Endpoint https://fetch.com/careers - Status Code: 200
Endpoint https://fetch.com/some/post/endpoint - Status Code: 403
Endpoint https://www.fetchrewards.com/ - Status Code: 200
fetch.com has 0% availability
www.fetchrewards.com has 0% availability
Endpoint https://fetch.com/ - Status Code: 200
Endpoint https://fetch.com/careers - Status Code: 200
Endpoint https://fetch.com/some/post/endpoint - Status Code: 403
Endpoint https://www.fetchrewards.com/ - Status Code: 200
fetch.com has 0% availability
www.fetchrewards.com has 0% availability
Endpoint https://fetch.com/ - Status Code: 200
Endpoint https://fetch.com/careers - Status Code: 200
Endpoint https://fetch.com/some/post/endpoint - Status Code: 403
Endpoint https://www.fetchrewards.com/ - Status Code: 200
fetch.com has 0% availability
www.fetchrewards.com has 0% availability
Endpoint https://fetch.com/ - Status Code: 200
Endpoint https://fetch.com/careers - Status Code: 200
Endpoint https

SystemExit: 0

In [ ]:
import requests
import yaml

# Sample YAML configuration data for a single endpoint
yaml_config = '''
- headers:
    user-agent: fetch-synthetic-monitor
  method: GET
  name: fetch index page
  url: https://fetch.com
'''

# Load YAML configuration
endpoint = yaml.safe_load(yaml_config)[0]  # Extract the first endpoint

# Extract endpoint details
url = endpoint.get('url')
method = endpoint.get('method', {})
headers = endpoint.get('headers', {})

try:
    if method == 'GET':
        response = requests.get(url, headers=headers)
    elif method == 'POST':
        # Add logic for POST method if needed
        pass
    else:
        raise ValueError('Invalid HTTP method')

    print(f"Request to {url} - Status Code: {response.status_code}")
    # Print response content if needed: print(response.json())

except requests.RequestException as e:
    print(f"Request to {url} failed: {e}")
